In [1]:
import numpy as np
import pandas as pd
from sqlalchemy import create_engine

import requests
import yfinance as yf

In [2]:
def get_all_tickers() -> str:
    url = 'https://quality.data.gov.tw/dq_download_json.php'
    num_id = '11549'
    md5_check = 'bb878d47ffbe7b83bfc1b41d0b24946e'
    r = requests.get(f'{url}?nid={num_id}&md5_url={md5_check}')
    return [i["證券代號"] for i in r.json()]

In [8]:
get_all_tickers()

JSONDecodeError: Expecting value: line 1 column 7 (char 6)

In [3]:
tickers = ['2330.TW', '0050.TW']#[ i+'.TW' for i in get_all_tickers() ]
stock = yf.download(
    tickers=tickers,
    start="2022-11-25",
    end="2022-12-05",
    interval="1d", # minute
    ignore_tz=False
)
stock

[*********************100%***********************]  2 of 2 completed


Adj Close               Close                High  \
                              0050.TW 2330.TW     0050.TW 2330.TW     0050.TW   
Date                                                                            
2022-11-25 00:00:00+08:00  116.699997   498.0  116.699997   498.0  116.949997   
2022-11-28 00:00:00+08:00  114.349998   480.5  114.349998   480.5  115.300003   
2022-11-29 00:00:00+08:00  115.400002   487.0  115.400002   487.0  115.400002   
2022-11-30 00:00:00+08:00  116.349998   490.0  116.349998   490.0  116.400002   
2022-12-01 00:00:00+08:00  118.349998   498.5  118.349998   498.5  119.500000   
2022-12-02 00:00:00+08:00  117.300003   492.5  117.300003   492.5  117.949997   

                                          Low                Open          \
                          2330.TW     0050.TW 2330.TW     0050.TW 2330.TW   
Date                                                                        
2022-11-25 00:00:00+08:00   498.0  116.349998   494.0  116.650002   494.0   
2022-11-28 00:00:00+08:00   490.0  113.900002   480.5  114.599998   486.0   
2022-11-29 00:00:00+08:00   488.0  113.099998   473.0  113.849998   473.5   
2022-11-30 00:00:00+08:00   490.0  114.849998   482.5  115.099998   483.0   
2022-12-01 00:00:00+08:00   508.0  118.000000   498.5  119.199997   506.0   
2022-12-02 00:00:00+08:00   497.0  117.000000   490.0  117.300003   490.0   

                             Volume            
                            0050.TW   2330.TW  
Date                                           
2022-11-25 00:00:00+08:00  16020486  39949969  
2022-11-28 00:00:00+08:00  15044738  38926919  
2022-11-29 00:00:00+08:00  10671467  38182197  
2022-11-30 00:00:00+08:00   9296066  57111603  
2022-12-01 00:00:00+08:00  24684339  41746878  
2022-12-02 00:00:00+08:00   7480152  28319679

In [4]:
def process_dataframe(stock):
    columns = [ 'Adj Close', 'Close', 'High', 'Low', 'Open', 'Volume' ]
    stocks = []
    for ticker in tickers:
        tmp_stock = stock[ [ (column, ticker) for column in columns] ].copy()
        tmp_stock.columns = columns
        tmp_stock['Ticker'] = ticker
        tmp_stock = tmp_stock[ ["Ticker"]+columns ]
        tmp_stock.reset_index(names='datetime', inplace=True)
        tmp_stock.dropna(subset=columns, how="all", axis=0, inplace=True)
        stocks += [ tmp_stock ]

    stock = pd.concat(stocks)
    stock.columns = [ ''.join(x.lower().split()) for x in stock.columns.tolist() ]
    return stock
stock = process_dataframe(stock)
stock.head()

,datetime,ticker,adjclose,close,high,low,open,volume
0,2022-11-25 00:00:00+08:00,2330.TW,498.0,498.0,498.0,494.0,494.0,39949969
1,2022-11-28 00:00:00+08:00,2330.TW,480.5,480.5,490.0,480.5,486.0,38926919
2,2022-11-29 00:00:00+08:00,2330.TW,487.0,487.0,488.0,473.0,473.5,38182197
3,2022-11-30 00:00:00+08:00,2330.TW,490.0,490.0,490.0,482.5,483.0,57111603
4,2022-12-01 00:00:00+08:00,2330.TW,498.5,498.5,508.0,498.5,506.0,41746878


In [5]:
from sqlalchemy.dialects.postgresql import insert


def _postgres_do_nothing_on_conflict(table, conn, keys, data_iter):
    print(type(table.table))
    print(table.table)
    data = [dict(zip(keys, row)) for row in data_iter]
    insert_statement = insert(table.table).values(data)
    upsert_statement = insert_statement.on_conflict_do_nothing(
        constraint=f"{table.table.name}_pkey"
    )
    conn.execute(upsert_statement)

dsn = "postgresql+psycopg2://admin:0000@db:5432/yahoo_stock"
engine = create_engine(dsn)

stock.to_sql(
    "yfinance",
    engine,
    if_exists="append",
    index=False,
    method=_postgres_do_nothing_on_conflict
)

<class 'sqlalchemy.sql.schema.Table'>
yfinance


In [6]:
pd.read_sql('select * from yfinance', engine)

,datetime,ticker,adjclose,close,high,low,open,volume,createdat,updatedat,deletedat
0,2022-11-24 16:00:00+00:00,2330.TW,498.000000,498.000000,498.000000,494.000000,494.000000,39949969,2022-12-05 10:16:56.545844+00:00,2022-12-05 10:16:56.545844+00:00,None
1,2022-11-27 16:00:00+00:00,2330.TW,480.500000,480.500000,490.000000,480.500000,486.000000,38926919,2022-12-05 10:16:56.545844+00:00,2022-12-05 10:16:56.545844+00:00,None
2,2022-11-28 16:00:00+00:00,2330.TW,487.000000,487.000000,488.000000,473.000000,473.500000,38182197,2022-12-05 10:16:56.545844+00:00,2022-12-05 10:16:56.545844+00:00,None
3,2022-11-29 16:00:00+00:00,2330.TW,490.000000,490.000000,490.000000,482.500000,483.000000,57111603,2022-12-05 10:16:56.545844+00:00,2022-12-05 10:16:56.545844+00:00,None
4,2022-11-30 16:00:00+00:00,2330.TW,498.500000,498.500000,508.000000,498.500000,506.000000,41746878,2022-12-05 10:16:56.545844+00:00,2022-12-05 10:16:56.545844+00:00,None
5,2022-12-01 16:00:00+00:00,2330.TW,492.500000,492.500000,497.000000,490.000000,490.000000,28319679,2022-12-05 10:16:56.545844+00:00,2022-12-05 10:16:56.545844+00:00,None
6,2022-11-24 16:00:00+00:00,0050.TW,116.699997,116.699997,116.949997,116.349998,116.650002,16020486,2022-12-05 10:16:56.545844+00:00,2022-12-05 10:16:56.545844+00:00,None
7,2022-11-27 16:00:00+00:00,0050.TW,114.349998,114.349998,115.300003,113.900002,114.599998,15044738,2022-12-05 10:16:56.545844+00:00,2022-12-05 10:16:56.545844+00:00,None
8,2022-11-28 16:00:00+00:00,0050.TW,115.400002,115.400002,115.400002,113.099998,113.849998,10671467,2022-12-05 10:16:56.545844+00:00,2022-12-05 10:16:56.545844+00:00,None
9,2022-11-29 16:00:00+00:00,0050.TW,116.349998,116.349998,116.400002,114.849998,115.099998,9296066,2022-12-05 10:16:56.545844+00:00,2022-12-05 10:16:56.545844+00:00,None


In [7]:
import os 
def create_project_init_structure(root_folder:str):
    __forder_list = ['configs', 'dao', 'init', 'utils', 'jobs']
    for dir_name in __forder_list:
        directory = os.path.join(
            root_folder, 
            dir_name)
        if not os.path.exists(directory):
            os.makedirs(directory)
            init_file_path = os.path.join(directory,'__init__.py')
            with open(init_file_path, 'w') as f:
                f.write("#!/usr/bin/python3")
        else:
            print(f'Already exists: {directory}')
create_project_init_structure("../")

Already exists: ../configs
Already exists: ../dao
Already exists: ../init
Already exists: ../utils
Already exists: ../jobs
